In [22]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound
from celmech.disturbing_function import laplace_b

In [23]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [46]:
def make_sim(alpha, mass, ecc, pom):
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)
    mu1, mu2, mu3 = mass
    ecc1, ecc2, ecc3 = ecc
    pomega1, pomega2, pomega3 = pom
    
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, e=ecc1, pomega=pomega1)
    sim.add(m=mu2, P=P2, e=ecc2, pomega=pomega2)
    sim.add(m=mu3, P=P3, e=ecc3, pomega=pomega3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    sim.dt = sim.particles[1].P/12
    sim.collision = "direct"
    
    print(ps[1].a/ps[2].a, ps[2].a/ps[3].a)
    
    return sim

In [70]:
def run(sim):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    print(M)
    
    return Mdiag

In [73]:
alpha = 0.8, 0.75 #0.735, 0.735
mass = 1e-7,1e-7,1e-7
ecc = 0.005, 0.005, 0.005
pomega = 0, 0, np.pi
sim = make_sim(alpha, mass, ecc, pomega)
run(sim)

0.7999999733333379 0.7499999750000065
[[-2.70306585e-06  2.01449690e-06  2.07274774e-07]
 [ 2.01449690e-06 -3.08101809e-06  7.64469447e-07]
 [ 2.07274774e-07  7.64469447e-07 -1.08703627e-06]]


array([[-4.96348180e-06, -2.64697796e-22,  0.00000000e+00],
       [-7.41153829e-22, -1.62327414e-06,  0.00000000e+00],
       [ 1.38966343e-22,  7.94093388e-23, -2.84364260e-07]])